In [5]:
####Codigo para crear nuevas columnas en el DF de terremotos
##Importamos pandas
import pandas as pd

##Cargamos el DF de terremotos cargando el fichero terremotos.csv.bz2
terremotos = pd.read_csv('../files/terremotos.csv.bz2',sep=';')

In [6]:
###Modificamos el nombre de varias columnas
terremotos.rename(columns={'Unnamed: 0':'ID','Time':'time_long','author':'agency_long',\
                           'region':'region_long','creationTime':'creationTime_long'},inplace=True)

In [11]:
##Importamos libreria math
import math

####Definicion de funciones para transformar las columnas


##Función que recoge fecha del terremoto en el formato del DF inicial, que incluye saltos de línea (\n). 
##Esta función separa por \n y devuelve fecha y hora del terremoto en este formato: "1998-01-01T21:33:56.2Z"
def separa_tiempo(cadena):
    lista = cadena.split('\n')
    return lista[1]


##Función que recoge fecha y hora en formato "1998-01-01T21:33:56.2Z" y devuelve la fecha en formato "yyyy-mm-dd":
##"1998-01-01"
def separa_fecha(cadena):
    cadena = str(cadena)
    if (len(cadena)!=3):
        lista = cadena.split('T')
        return lista[0]

    
##Función que recoge fecha y hora en formato "1998-01-01T21:33:56.2Z" y devuelve la hora en formato "hh-mm-ss":
##"21:33:56"
def separa_hora(cadena):
    cadena = str(cadena)
    if (len(cadena)!=3):
        lista = cadena.split('T')
        hora = lista[1].split('.')
        return hora[0]

    
##Función que recoge fecha en formato "1998-01-01" y devuelve el año "1998":
def dime_anio(cadena):
    lista = cadena.split('-')
    return lista[0]


##Función para obtener la agencia que detecta el terremoto. Recibe el formato "smi:smi-registry/organization/EMSC"
##y retorna "EMSC"
def obtener_author(cadena):
    cadena = str(cadena)
    #Caso concreto NaN
    if(len(cadena)==3):
        ###Solo hay 6 NaN por lo que los asignamos al grupo mas numeroso, EMSC
        return 'EMSC'
    ##Caso genérico
    else:
        lista = cadena.split('/')
        return lista[-1]

    
##Función que recoge la región del terremoto en el formato del DF inicial, que incluye saltos de línea (\n). 
##Esta función separa por \n. Recibe la región en formato "\nADRIATIC SEA\nFlinn-Engdahl region\n" 
##y retorna "ADRIATIC SEA"
def separa_regiones(cadena):
    lista = cadena.split('\n')
    return lista[1]


##Función para extraer el valor entero de las coordenadas, tanto para longitud como latitud. Por ejemplo, si recibe
##un valor de latitud 37.5460, retorna 37. Para valores de 180 y -180, retorna 179 y -179 respectivamente.
def coordenada_entera(decimal):
    decimal = str(decimal)
    lista = decimal.split('.')
    if (lista[0]=='180'):
        lista[0]='179'
    if (lista[0]=='-180'):
        lista[0]='-179'
    return lista[0]

##Algunas regiones tienen espacios en blanco al principio y al final. Esta función elimina esos espacios
def elimina_espacios(cadena):
    return cadena.strip()

In [12]:
##La función mas compleja es la extracción del país donde se produce el terremoto. Para extraer el país usaremos el
##campo region. Directamente, en muchos casos tenemos el nombre del país en el campo region; así que primero extraemos 
##el nombre del país en estos casos; después, tenemos muchos campos region que son estados USA, en este caso asignamos
##'UNITED STATES OF AMERICA'. Luego, y en los casos que en region aparecen las palabras 'OCEAN' y 'SEA', 
##asignaremos 'SEA'. Después, tenemos 170 valores de región que no se corresponden con ninguno de los casos anteriores;
##para asignar el país, creamos un diccionario que relaciona region y país. Y finalmente, dos casos específicos;
##el primero para 31 registros "D’ENTRECASTEAUX ISLANDS" asignamos 'PAPUA NEW GUINEA'; el segundo para aquellos con
##el campo region vacío '', que corresponden a 'ITALY'


###Cargamos fichero 'paises.csv' con el nombre de todos los países del mundo y convertimos el nombre de los países 
###en una lista Python con los nombres en mayúsculas
paises = pd.read_csv('../files/paises.csv', sep=';')
paises.columns = paises.columns.str.strip()
listado_paises = paises['name'].str.upper()
listado_paises = listado_paises.tolist()

###Cargamos fichero 'estados_usa.csv' con el nombre de todos los estados USA y convertimos el nombre de los estados en
###una lista Python con los nombres en mayúsculas
estados_usa = pd.read_csv('../files/estados_usa.csv', sep=';')
estados_usa.columns = estados_usa.columns.str.strip()
listado_estados = estados_usa['state'].str.upper()
listado_estados = listado_estados.tolist()

###Cargamos fichero 'regiones_sin_nombre_pais.csv' que relaciona region:pais y lo convierte en una lista Python
regiones_paises = pd.read_csv('../files/regiones_sin_nombre_pais.csv', sep=';')
regiones_paises = regiones_paises.values.tolist()

##Creamos diccionario vacio
diccionario_paises={}

##Bucle para convertir la lista region:pais en un diccionario
for valor in regiones_paises:
    key = valor[0]
    key = key.lstrip()
    value = valor[1]
    diccionario_paises[key]=value

    
###Funcion para extraer el nombre del pais
def obten_pais (cadena):
    #Extraccion del nombre del pais en el campo region
    for i in listado_paises:
        if (i in cadena):
            return i
    #Relacion con UNITED STATES OF AMERICA de los estados USA
    for j in listado_estados:
        if (j in cadena):
            return 'UNITED STATES OF AMERICA'
    #Relacion de SEA con mar
    if ('SEA' in cadena):
        return 'SEA'
    #Relacion de OCEAN con mar
    if ('OCEAN' in cadena):
        return 'SEA'
    #Extraccion del pais en base al diccionario de regiones
    if (cadena in diccionario_paises.keys()):
        return diccionario_paises[cadena]
    #Casos especificos
    if ('ENTRECASTEAUX' in cadena):
        return 'PAPUA NEW GUINEA'
    if (cadena==''):
        return 'ITALY'


In [13]:
####Creacion de columnas adicionales en el DataFrame de terremotos

###Columna para modificar formato fecha. De "\n1998-01-01T21:33:56.2Z\n0.44\n" a "1998-01-01T21:33:56.2Z"
terremotos['time1_long'] = terremotos['time_long'].apply(separa_tiempo)

#Columna para obtener la fecha del terremoto. De "1998-01-01T21:33:56.2Z" a "1998-01-01"
terremotos['date'] = terremotos['time1_long'].apply(separa_fecha)

#Columna para obtener la hora del terremoto. De "1998-01-01T21:33:56.2Z" a "21:33:56"
terremotos['hour'] = terremotos['time1_long'].apply(separa_hora)

#Columna para obtener el año del terremoto. De "1998-01-01" a "1998"
terremotos['year'] = terremotos['date'].apply(dime_anio)

#Columna para obtener la agencia que detecta el terremoto
terremotos['agency'] = terremotos['agency_long'].apply(obtener_author)

#Columna para extraer la fecha de inclusión en la web del terremoto. Formato "1998-01-01"
terremotos['creation_date'] = terremotos['creationTime_long'].apply(separa_fecha)

#Columna para extraer la hora de inclusión en la web del terremoto. Formato "21:33:56"
terremotos['creation_hour'] = terremotos['creationTime_long'].apply(separa_hora)

#Columna para dar formato al campo region.De "\nADRIATIC SEA\nFlinn-Engdahl region\n" a "ADRIATIC SEA"
terremotos['region'] = terremotos['region_long'].apply(separa_regiones)
terremotos['region'] = terremotos['region'].apply(elimina_espacios)

#Columna para extraer el valor entero de la latitud. De 37.12 a 37
terremotos['latitud_entero'] = terremotos['latitude'].apply(coordenada_entera)

#Columna para extraer el valor entero de la longitud. De 37.12 a 37
terremotos['longitud_entero'] = terremotos['longitude'].apply(coordenada_entera)

#Columna para juntar los valores enteros de latitud y longitud
terremotos['coordenadas'] = (terremotos['latitud_entero'].apply(str)) + ',' + (terremotos['longitud_entero'].apply(str))

#Columna con el pais a partir de la columna region
terremotos['paises'] = terremotos['region'].apply(obten_pais)

In [31]:
###Guardamos el fichero de terremotos con todas las columnas como "terremotos_long.csv"
terremotos.to_csv('terremotos_long.csv',sep=';')